In [1]:
###Libraries
import json
import random
import sys
import numpy as np

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using cuDNN version 5103 on context None
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 140, in init_dev
    context.free_gmem / MB))
RuntimeError: Trying to preallocate 10867 MB of GPU memory while only 237 MB are available.
Using Theano backend.


In [3]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [6]:
path = "data/state-farm/"
train_data = get_data(path+'train')
val_data = get_data(path+'valid')
test_data = get_data(path+'test')

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [7]:
class QuadraticCost(object):
    @staticmethod
    def fn(a,y):
        return 0.5*np.linalg.norm(y-a)**2
    
    @staticmethod
    def delta(z,a,y):
        """return error delta"""
        return (a-y)*sigmoid_prime(z)
    
class CrossEntropyCost(object):
    @staticmethod
    def fn(a,y):
        return -(1/n) * np.sum(-y*np.log(a) + (1-y)*np.log(1-a))

class Network(object):
    def __init__(self, sizes):
        """
        sizes is the same as in Nielsen's implementation, where it is a vector
        containing the number of neurons in respective layers of the network
        e.g. [4,3,7] is a 3-layer network with 4 neurons in the first layer, 3 in the second, and so on. 
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for
                       x,y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self,a):
        """determine output of network if input is a"""
        for b,w in zip(self.biases,self.weights):
            a = sigmoid(np.dot(w,a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Train network using mini-batch stochastic gradient descent
        """
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range [0,n,mini_batch_size]
            ]
            for mini_batch in mini_batches:
                self = self.update_mini_batch(mini_batch,eta)
    
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x,y in mini_batch:
            #compute change in nabla_b/w from backpropagation
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            #update nabla_b and nabla_w
            nabla_b = [nb+dnb for nb,dnb in zip(nabla_b,delta_nabla_b)]
            nabla_w = [dw+dnw for nw,dnw in zip(nabla_w,delta_nabla_w)]
            
        self.weights = [w - (eta/len(mini_batch))*nw for w,nw in zip(self.weights,nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb for b,nb in zip(self.biases,nabla_b)]
        
    def backprop(self,x,y):
        """
        returns a tuple (nabla_b,nabla_w) representing the gradient for the cost function C_x
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        #feedforwared
        activation = x
        activations = [x] #list to store activations layer by layer
        
        zs = [] #list to store zs (weighted input to neurons in layer l)
        
        for b,w in zip(self.biases,self.weights):
            z = np.dot(w,activation) + b
            zs.append(z)
            
            activation = sigmoid(z)
            activations.append(activation)
            
        for l in range(2,self.num_layers):
            z = zs[-1]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[l-1].transpose(),delta) * sp
            
            #according to the third equation of backpropagation, dC/db = delta so we make the last entry in nabla_b delta
            nabla_b[-1] = delta
            #the fourth equation of backpropagation gives dC/dw where this w is the from the kth row to the jth
            nabla_w[-1] = np.dot(activations[l-1], delta)
        return (nabla_b,nabla_w)
    
    def evaluate(self, test_data):
        """
        return number of inputs for which the neural network gives the correct output
        """
        test_results = [np.argmax(feedforward(x),y) for x,y in test_data]
        return sum(int(x==y) for (x,y) in test_results)
    

##helper functions##
def sigmoid(z):
    """The sigmoid function"""
    return 1.0/(1+np.exp(-z))
def sigmoid_prime(z):
    """derivative of the sigmoid function"""
    return sigmoid(x)*(1-sigmoid(z))

In [10]:
net = Network([784,30,10]) #create a network w 30 hidden neurons
net.SGD(train_data,5,16,3.0,test_data=test_data) #train w SGD, 5 epochs, batchsize = 16, learning rate = 3.0

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()